In [1]:
import threading
import os
import pandas as pd
import numpy as np
import eikon as ek

ek.set_app_key('cec2abe836754fe692a01de15032769539511a71')
import time
from datetime import datetime

from uti import Logger, DataLoader
from Broker import get_tr, get_atr
from Path import DATABASE_PATH

DL = DataLoader()
logger = Logger()
lock = threading.RLock()

NOW_STR = logger.NOW_STR
testing = False


C:\Users\JayYang\Platform\Citi\Path\settings.py C:\Users\JayYang\Platform\Citi\Path\settings.py
Base path C:/Users/JayYang
Platform path C:/Users/JayYang/Platform
Onedrive directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Daily path C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Daily
Database path C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/Citi


In [2]:
from Eikon import Eikon_update_price

In [3]:
ticker_df = DL.loadDB('ticker_to_ISIN_20220525.csv')
ticker_df.head()

17:54:14 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/Citi\Log/20220525/20220525175250.txt created.
17:54:14 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/Citi\ticker_to_ISIN_20220525.csv


,ID,ISIN
0,ALSEN FP Equity,FR0012596468
1,DTG SW Equity,DE000DTR0CK8
2,EKI FP Equity,FR0011466069
3,GSK LN Equity,GB0009252882
4,IVA FP Equity,FR0013233012


In [24]:
ticker_df = DL.loadDB('tickers universe to check.csv')
ticker_df = ticker_df.drop_duplicates(['Ticker']).reset_index(drop=True)
ticker_df['ISIN'] = ''

19:13:54 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/tickers universe to check.csv


In [13]:
ticker_df

,Ticker(old),Ticker,Ticker(BBG),Region
0,2432,2432.T,2432 JP,JP
1,2433,2433.T,2433 JP,JP
2,2670,2670.T,2670 JP,JP
3,2768,2768.T,2768 JP,JP
4,3279,3279.T,3279 JP,JP
...,...,...,...,...
6342,5101,5101.T,5101 JP,JP
6343,4689,4689.T,4689 JP,JP
6344,9474,9474.T,9474 JP,JP
6345,4205,4205.T,4205 JP,JP


In [6]:
ek.get_data([ticker_df['Ticker'][0]], 'TR.ISIN')[0]['ISIN'][0]

,Instrument,ISIN
0,2432.T,JP3548610009


In [10]:
result = ek.get_data(['testing'], 'TR.ISIN')[0]['ISIN'][0]
result

<NA>

In [ ]:
'''for i, row in ticker_df.iloc[:10].iterrows():
    ticker = row['Ticker']
    isin_code = ek.get_data([ticker], 'TR.ISIN')[0]['ISIN'][0]
    if not pd.isna(isin_code):
        ticker_df.loc[i, 'ISIN'] = isin_code'''

In [34]:
isin_code = ek.get_data(list(ticker_df['Ticker']), 'TR.ISIN')
isin_code[0]['ISIN']

0       JP3548610009
1       JP3766550002
2       JP3152740001
3       JP3663900003
4       JP3047490002
            ...     
5710    JP3931600005
5711    JP3955000009
5712    JP3955800002
5713    JP3430400006
5714    JP3826520003
Name: ISIN, Length: 5715, dtype: string

In [38]:
ticker_df['ISIN'] = isin_code[0]['ISIN']

In [39]:
DL.toDB(ticker_df, 'ticker universe with ISIN.csv')

In [53]:
result = ek.get_data(['JP3826520003'], ['TR.GICSSector', 'TR.GICSIndustry', 'TR.GICSIndustryGroup', 'TR.GICSSubIndustry',
                                  'CF_EXCHANGE', 'TR.ExchangeCountry', 'TR.UltimateParentCountryHQ'])[0]
result

,Instrument,RIC,CF_NAME,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CURRENCY,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,JP3826520003,4478.T,<NA>,Information Technology,Software,Software & Services,Application Software,<NA>,<NA>,Japan,Japan


In [54]:
result = ek.get_data(['4478.T'], ['CF_NAME', 'CURRENCY'])[0]
result

,Instrument,RIC,CF_NAME,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CURRENCY,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,4478.T,4478.T,FREEE/d,Information Technology,Software,Software & Services,Application Software,JPY,<NA>,Japan,Japan


In [8]:
ticker_df = DL.loadDB('ticker universe with ISIN.csv')

16:57:58 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/Log/20220427/20220427152757.txt created.
16:57:58 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/ticker universe with ISIN.csv


In [9]:
bbg_df = DL.loadDB('isin_ticker_dict.csv')

17:02:16 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/isin_ticker_dict.csv


In [12]:
bbg_df = bbg_df.rename(columns={'ID':'ISIN', 'ID_SECURITY_DES()': 'Ticker(BBG)'})
bbg_df_dict = bbg_df.set_index('ISIN').to_dict()['Ticker(BBG)']

{'SA0007879097': 'SAMBA AB Equity',
 'SA15DHKGHBH4': 'TADAWULG AB Equity',
 'SA0007879154': 'SCERCO AB Equity',
 'SA15CGS10H12': 'ACWA AB Equity',
 'AU0000157067': '29M AU Equity',
 'SA1510P1UMH1': 'SULAIMAN AB Equity',
 'SA000A0MWH44': 'BUDGET AB Equity',
 'DE000TRAT0N7': '8TRA GR Equity',
 'IT0001233417': 'A2A IM Equity',
 'ES0109427734': 'A3M SM Equity',
 'DE000A2DAM03': 'AAG GR Equity',
 'ID1000066004': 'AALI IJ Equity',
 'AU000000ABC7': 'ABC AU Equity',
 'AU000000ABP9': 'ABP AU Equity',
 'PHY0486V1154': 'AC PM Equity',
 'CA0089118776': 'AC CN Equity',
 'NO0010890304': 'ACC NO Equity',
 'ID1000125503': 'ACES IJ Equity',
 'ES0132105018': 'ACX SM Equity',
 'INE647O01011': 'ABFRL IN Equity',
 'AT000ADDIKO0': 'ADKO AV Equity',
 'AEA006101017': 'ADNOCDIS UH Equity',
 'ID1000111305': 'ADRO IJ Equity',
 'NL0000303709': 'AG SW Equity',
 'BRAERIACNOR4': 'AERI3 BZ Equity',
 'CL0002694637': 'AESANDES CI Equity',
 'AU000000AFG9': 'AFG AU Equity',
 'RU0009062285': 'AFLT RM Equity',
 'BE09742649

In [18]:
sum(ticker_df['ISIN'].replace(bbg_df_dict).isna())

230

In [23]:
ticker_df.loc[ticker_df['Ticker(BBG)'].isna(), 'Ticker(BBG)'] = ticker_df.loc[ticker_df['Ticker(BBG)'].isna(), 'ISIN'].replace(bbg_df_dict)

In [26]:
ticker_df[ticker_df['Ticker(BBG)'].isna()]

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN
40,000418.SZ,000418.SZ,000418 CH,CH,NaN
356,0958.HK,0958.HK,958 HK,HK,NaN
1224,688235.SS,688235.SS,688235 CH,CH,NaN
1225,688339.SS,688339.SS,688339 CH,NaN,NaN
1229,688686.SS,688686.SS,688686 CH,CH,NaN
...,...,...,...,...,...
5557,WPG,WPG.K,WPG US,US,NaN
5563,WPX,WPX.K,WPX US,US,NaN
5587,XEC,XEC,XEC US,US,NaN
5628,ZAYO,ZAYO.K,ZAYO US,US,NaN


In [28]:
DL.toDB(ticker_df, 'ticker universe with ISIN.csv')

In [41]:
ticker_df_valid = ticker_df[~ticker_df['ISIN'].isna()].reset_index(drop=True)
ticker_df_invalid = ticker_df[ticker_df['ISIN'].isna()].reset_index(drop=True)

In [42]:
RIC = ek.get_data(list(ticker_df_valid['ISIN']), 'TR.RIC')[0]

In [44]:
ticker_df_valid['RIC'] = RIC['RIC']

In [46]:
ticker_df_valid[ticker_df_valid['RIC'] != ticker_df_valid['Ticker']]

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC
42,0005.HK,0005.HK,5 HK,HK,GB0005405286,HSBA.L
273,0486.HK,0486.HK,486 HK,HK,RU000A1025V3,RUAL.MM
275,0494.HK,0494.HK,494 HK,HK,BMG5485F1692,0494.HK^E20
378,1090.SE,1090.SE,SAMBA AB Equity,NaN,SA0007879097,1090.SE^D21
393,1169.HK,1169.HK,1169 HK,HK,BMG423131256,HRELF.PK^A21
...,...,...,...,...,...,...
5424,ZVIA,ZVIA.K,ZVIA US,CN,US98955K1043,ZVIA.N
5425,ZWS,ZWS,ZWS US,US,US98983L1089,ZWS.N
5426,ZY,ZY.O,ZY US,US,US98985X1000,ZY.OQ
5427,ZYDU.BO,ZYDU.BO,ZYDUSLIF IN Equity,IN,INE010B01027,ZYDU.NS


In [55]:
sectors = ek.get_data(list(ticker_df_valid['ISIN']), 
                      ['TR.GICSSector', 'TR.GICSIndustry', 'TR.GICSIndustryGroup', 'TR.GICSSubIndustry',
                      'CF_EXCHANGE', 'TR.ExchangeCountry', 'TR.UltimateParentCountryHQ'])[0]

In [59]:
ticker_df_valid = pd.concat([ticker_df_valid, 
                               sectors[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                        'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']]], axis=1)

In [60]:
ticker_df_valid

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,2432,2432.T,2432 JP,JP,JP3548610009,2432.T,Communication Services,Entertainment,Media & Entertainment,Interactive Home Entertainment,<NA>,Japan,Japan
1,2433,2433.T,2433 JP,JP,JP3766550002,2433.T,Communication Services,Media,Media & Entertainment,Advertising,<NA>,Japan,Japan
2,2670,2670.T,2670 JP,JP,JP3152740001,2670.T,Consumer Discretionary,Specialty Retail,Retailing,Apparel Retail,<NA>,Japan,Japan
3,2768,2768.T,2768 JP,JP,JP3663900003,2768.T,Industrials,Trading Companies & Distributors,Capital Goods,Trading Companies & Distributors,<NA>,Japan,Japan
4,3279,3279.T,3279 JP,JP,JP3047490002,3279.T,Real Estate,Equity Real Estate Investment Trusts (REITs),Real Estate,Diversified REITs,<NA>,Japan,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480,2267,2267.T,2267 JP,JP,JP3931600005,2267.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,<NA>,Japan,Japan
5481,6841,6841.T,6841 JP,JP,JP3955000009,6841.T,Information Technology,"Electronic Equipment, Instruments & Components",Technology Hardware & Equipment,Electronic Equipment & Instruments,<NA>,Japan,Japan
5482,5101,5101.T,5101 JP,JP,JP3955800002,5101.T,Consumer Discretionary,Auto Components,Automobiles & Components,Tires & Rubber,<NA>,Japan,Japan
5483,9474,9474.T,9474 JP,JP,JP3430400006,9474.T,Communication Services,Media,Media & Entertainment,Publishing,<NA>,Japan,Japan


In [61]:
DL.toDB(ticker_df_valid, 'ticker_df_valid.csv')
DL.toDB(ticker_df_invalid, 'ticker_df_invalid.csv')

In [66]:
ticker_from_sunny = DL.loadDB('Datasets/ticker_unique.csv')

19:31:38 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/Datasets/ticker_unique.csv


In [70]:
ticker_from_sunny=ticker_from_sunny.drop_duplicates(['1'])

In [71]:
DL.toDB(ticker_from_sunny, 'ticker_unique.csv')

In [72]:
ticker_from_sunny = DL.loadDB('ticker_to_ISIN_1.csv')

00:41:04 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/ticker_to_ISIN_1.csv


In [73]:
ticker_from_sunny = ticker_from_sunny[~ticker_from_sunny['ISIN'].isna()]

In [77]:
ticker_from_sunny_dict = ticker_from_sunny.set_index('ISIN').to_dict()['ID']

In [81]:
ticker_df_valid.loc[ticker_df_valid['ISIN'].isin(ticker_from_sunny['ISIN']), 'Ticker(BBG)'] = ticker_df_valid[ticker_df_valid['ISIN'].isin(ticker_from_sunny['ISIN'])]['ISIN'].replace(ticker_from_sunny_dict)

In [82]:
ticker_df_valid.loc[ticker_df_valid['ISIN'].isin(ticker_from_sunny['ISIN']), 'Ticker(BBG)']

1       2433 JP Equity
2       2670 JP Equity
6       3402 JP Equity
8       4004 JP Equity
9       4005 JP Equity
             ...      
5461    4516 JP Equity
5462    4021 JP Equity
5465    4716 JP Equity
5469    4565 JP Equity
5484    4478 JP Equity
Name: Ticker(BBG), Length: 2995, dtype: object

In [83]:
ticker_df_valid

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,2432,2432.T,2432 JP,JP,JP3548610009,2432.T,Communication Services,Entertainment,Media & Entertainment,Interactive Home Entertainment,<NA>,Japan,Japan
1,2433,2433.T,2433 JP Equity,JP,JP3766550002,2433.T,Communication Services,Media,Media & Entertainment,Advertising,<NA>,Japan,Japan
2,2670,2670.T,2670 JP Equity,JP,JP3152740001,2670.T,Consumer Discretionary,Specialty Retail,Retailing,Apparel Retail,<NA>,Japan,Japan
3,2768,2768.T,2768 JP,JP,JP3663900003,2768.T,Industrials,Trading Companies & Distributors,Capital Goods,Trading Companies & Distributors,<NA>,Japan,Japan
4,3279,3279.T,3279 JP,JP,JP3047490002,3279.T,Real Estate,Equity Real Estate Investment Trusts (REITs),Real Estate,Diversified REITs,<NA>,Japan,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480,2267,2267.T,2267 JP,JP,JP3931600005,2267.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,<NA>,Japan,Japan
5481,6841,6841.T,6841 JP,JP,JP3955000009,6841.T,Information Technology,"Electronic Equipment, Instruments & Components",Technology Hardware & Equipment,Electronic Equipment & Instruments,<NA>,Japan,Japan
5482,5101,5101.T,5101 JP,JP,JP3955800002,5101.T,Consumer Discretionary,Auto Components,Automobiles & Components,Tires & Rubber,<NA>,Japan,Japan
5483,9474,9474.T,9474 JP,JP,JP3430400006,9474.T,Communication Services,Media,Media & Entertainment,Publishing,<NA>,Japan,Japan


In [84]:
ticker_from_sunny_tbu = ticker_from_sunny[~ticker_from_sunny['ISIN'].isin(ticker_df_valid['ISIN'])]

In [89]:
ticker_from_sunny_tbu = ticker_from_sunny_tbu.drop(['Partial Errors'], axis=1)
ticker_from_sunny_tbu = ticker_from_sunny_tbu.rename(columns={'ID': 'Ticker(BBG)'})


,Ticker(BBG),ISIN
0,2020 NO Equity,BMG9156K1018
2,ACAD SS Equity,SE0007897079
3,ACAST SS Equity,SE0015960935
4,ACROUD SS Equity,SE0001863291
5,ANODB SS Equity,SE0000472268
...,...,...
5178,AND CN Equity,CA0342231077
5179,AKU CN Equity,CA01021F1099
5180,AD-U CN Equity,CAC010971017
5181,ACO/X CN Equity,CA0467894006


In [90]:
ticker_from_sunny_tbu = ticker_from_sunny_tbu.reset_index(drop=True)

In [91]:
sectors2 = ek.get_data(list(ticker_from_sunny_tbu['ISIN']), 
                      ['TR.GICSSector', 'TR.GICSIndustry', 'TR.GICSIndustryGroup', 'TR.GICSSubIndustry',
                      'CF_EXCHANGE', 'TR.ExchangeCountry', 'TR.UltimateParentCountryHQ'])[0]

In [92]:
ticker_from_sunny_tbu = pd.concat([ticker_from_sunny_tbu, 
                                   sectors2[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                            'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']]], axis=1)

In [93]:
ticker_from_sunny_tbu

,Ticker(BBG),ISIN,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,2020 NO Equity,BMG9156K1018,Industrials,Marine,Transportation,Marine,<NA>,Norway,Bermuda
1,ACAD SS Equity,SE0007897079,Consumer Discretionary,Diversified Consumer Services,Consumer Services,Education Services,<NA>,Sweden,Sweden
2,ACAST SS Equity,SE0015960935,Communication Services,Interactive Media & Services,Media & Entertainment,Interactive Media & Services,<NA>,Sweden,Sweden
3,ACROUD SS Equity,SE0001863291,Consumer Discretionary,"Hotels, Restaurants & Leisure",Consumer Services,Casinos & Gaming,<NA>,Sweden,Sweden
4,ANODB SS Equity,SE0000472268,Information Technology,IT Services,Software & Services,IT Consulting & Other Services,<NA>,Sweden,Sweden
...,...,...,...,...,...,...,...,...,...
2208,AND CN Equity,CA0342231077,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,Canada,Canada
2209,AKU CN Equity,CA01021F1099,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,United States of America,United States
2210,AD-U CN Equity,CAC010971017,Financials,Capital Markets,Diversified Financials,Asset Management & Custody Banks,<NA>,Canada,Canada
2211,ACO/X CN Equity,CA0467894006,Utilities,Multi-Utilities,Utilities,Multi-Utilities,<NA>,Canada,Canada


In [94]:
RIC = ek.get_data(list(ticker_from_sunny_tbu['ISIN']), 'TR.RIC')[0]
ticker_from_sunny_tbu['RIC'] = RIC['RIC']

In [100]:
ticker_from_sunny_tbu['Ticker(old)'] = ticker_from_sunny_tbu['RIC']
ticker_from_sunny_tbu['Ticker'] = ticker_from_sunny_tbu['RIC']
ticker_from_sunny_tbu['Region'] = ''
ticker_from_sunny_dict = ticker_from_sunny.set_index('ISIN').to_dict()['ID']
ticker_from_sunny_tbu['Ticker(BBG)'] = ticker_from_sunny_tbu['ISIN'].replace(ticker_from_sunny_dict)

In [101]:
ticker_from_sunny_tbu['Region'] = ticker_from_sunny_tbu['Ticker(BBG)'].str.split(' ', 1)

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,20202.OL,20202.OL,20202.OL,,BMG9156K1018,20202.OL,Industrials,Marine,Transportation,Marine,<NA>,Norway,Bermuda
1,ACADE.ST,ACADE.ST,ACADE.ST,,SE0007897079,ACADE.ST,Consumer Discretionary,Diversified Consumer Services,Consumer Services,Education Services,<NA>,Sweden,Sweden
2,ACAST.ST,ACAST.ST,ACAST.ST,,SE0015960935,ACAST.ST,Communication Services,Interactive Media & Services,Media & Entertainment,Interactive Media & Services,<NA>,Sweden,Sweden
3,ACROUD.ST,ACROUD.ST,ACROUD.ST,,SE0001863291,ACROUD.ST,Consumer Discretionary,"Hotels, Restaurants & Leisure",Consumer Services,Casinos & Gaming,<NA>,Sweden,Sweden
4,ANODb.ST,ANODb.ST,ANODb.ST,,SE0000472268,ANODb.ST,Information Technology,IT Services,Software & Services,IT Consulting & Other Services,<NA>,Sweden,Sweden
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,AND.TO,AND.TO,AND.TO,,CA0342231077,AND.TO,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,Canada,Canada
2209,AKU.OQ,AKU.OQ,AKU.OQ,,CA01021F1099,AKU.OQ,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,United States of America,United States
2210,AD_u.TO,AD_u.TO,AD_u.TO,,CAC010971017,AD_u.TO,Financials,Capital Markets,Diversified Financials,Asset Management & Custody Banks,<NA>,Canada,Canada
2211,ACOx.TO,ACOx.TO,ACOx.TO,,CA0467894006,ACOx.TO,Utilities,Multi-Utilities,Utilities,Multi-Utilities,<NA>,Canada,Canada


In [116]:
ticker_from_sunny_tbu['Region'] = ticker_from_sunny_tbu['Ticker(BBG)'].str.split(' ', 0).apply(lambda x: x[1])

In [117]:
ticker_from_sunny_tbu[ticker_df_valid.columns]

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,20202.OL,20202.OL,2020 NO Equity,NO,BMG9156K1018,20202.OL,Industrials,Marine,Transportation,Marine,<NA>,Norway,Bermuda
1,ACADE.ST,ACADE.ST,ACAD SS Equity,SS,SE0007897079,ACADE.ST,Consumer Discretionary,Diversified Consumer Services,Consumer Services,Education Services,<NA>,Sweden,Sweden
2,ACAST.ST,ACAST.ST,ACAST SS Equity,SS,SE0015960935,ACAST.ST,Communication Services,Interactive Media & Services,Media & Entertainment,Interactive Media & Services,<NA>,Sweden,Sweden
3,ACROUD.ST,ACROUD.ST,ACROUD SS Equity,SS,SE0001863291,ACROUD.ST,Consumer Discretionary,"Hotels, Restaurants & Leisure",Consumer Services,Casinos & Gaming,<NA>,Sweden,Sweden
4,ANODb.ST,ANODb.ST,ANODB SS Equity,SS,SE0000472268,ANODb.ST,Information Technology,IT Services,Software & Services,IT Consulting & Other Services,<NA>,Sweden,Sweden
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,AND.TO,AND.TO,AND CN Equity,CN,CA0342231077,AND.TO,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,Canada,Canada
2209,AKU.OQ,AKU.OQ,AKU CN Equity,CN,CA01021F1099,AKU.OQ,Health Care,Health Care Providers & Services,Health Care Equipment & Services,Health Care Services,<NA>,United States of America,United States
2210,AD_u.TO,AD_u.TO,AD-U CN Equity,CN,CAC010971017,AD_u.TO,Financials,Capital Markets,Diversified Financials,Asset Management & Custody Banks,<NA>,Canada,Canada
2211,ACOx.TO,ACOx.TO,ACO/X CN Equity,CN,CA0467894006,ACOx.TO,Utilities,Multi-Utilities,Utilities,Multi-Utilities,<NA>,Canada,Canada


In [118]:
ticker_df_valid = pd.concat([ticker_df_valid, ticker_from_sunny_tbu[ticker_df_valid.columns]], axis=0)

In [120]:
DL.toDB(ticker_df_valid, 'ticker_df_final.csv')

In [121]:
mizuho_tickers = DL.loadDB('Datasets/Mizuho tickers.csv')

10:56:35 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/Datasets/Mizuho tickers.csv


In [171]:
mizuho_tickers_df = mizuho_tickers.drop_duplicates(['Ticker(BBG)'])
mizuho_tickers_df = mizuho_tickers_df.drop(['0'], axis=1)

In [172]:
mizuho_tickers_df = mizuho_tickers_df.reset_index(drop=True)

In [175]:
mizuho_tickers_dict = sectors3.set_index('Instrument').to_dict()['ISIN']
mizuho_tickers_df['ISIN'] = ''
mizuho_tickers_df['ISIN'] = mizuho_tickers_df['Ticker'].replace(mizuho_tickers_dict)
mizuho_tickers_df = mizuho_tickers_df[~mizuho_tickers_df['ISIN'].isin(ticker_df_valid['ISIN'])].reset_index(drop=True)
mizuho_tickers_df = mizuho_tickers_df[mizuho_tickers_df['ISIN']!= ''].reset_index(drop=True)
mizuho_tickers_df

,Ticker(BBG),Ticker,Ticker(old),ISIN
0,8977 JP Equity,8977.T,8977.T,JP3046320002
1,3397 JP Equity,3397.T,3397.T,JP3636650008
2,7718 JP Equity,7718.T,7718.T,JP3399000003
3,5463 JP Equity,5463.T,5463.T,JP3871200006
4,6622 JP Equity,6622.T,6622.T,JP3497800007
...,...,...,...,...
209,3227 JP Equity,3227.T,3227.T,JP3046450007
210,6866 JP Equity,6866.T,6866.T,JP3783200003
211,9793 JP Equity,9793.T,9793.T,JP3485600005
212,6804 JP Equity,6804.T,6804.T,JP3845800006


In [174]:
sectors3 = ek.get_data(list(mizuho_tickers_df['Ticker']), ['TR.ISIN'])[0]
sectors3 = sectors3[~sectors3.isna()]
sectors3[~sectors3['ISIN'].isin(ticker_df_valid['ISIN'])]

,Instrument,ISIN
0,8977.T,JP3046320002
1,3397.T,JP3636650008
2,7718.T,JP3399000003
3,5463.T,JP3871200006
4,6622.T,JP3497800007
...,...,...
209,3227.T,JP3046450007
210,6866.T,JP3783200003
211,9793.T,JP3485600005
212,6804.T,JP3845800006


In [178]:
sectors4 = ek.get_data(list(mizuho_tickers_df['ISIN']), 
                      ['TR.GICSSector', 'TR.GICSIndustry', 'TR.GICSIndustryGroup', 'TR.GICSSubIndustry',
                      'CF_EXCHANGE', 'TR.ExchangeCountry', 'TR.UltimateParentCountryHQ'])[0]

mizuho_tickers_df = pd.concat([mizuho_tickers_df, 
                               sectors4[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                         'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']]], axis=1)

In [189]:
mizuho_tickers_df['RIC'] = ''
mizuho_tickers_df['Region'] = ''
mizuho_tickers_df['Region'] = mizuho_tickers_df['Ticker(BBG)'].str.split(' ', 0).apply(lambda x: x[1])

RIC = ek.get_data(list(mizuho_tickers_df['ISIN']), 'TR.RIC')[0]
mizuho_tickers_df['RIC'] = RIC['RIC']

In [185]:
DL.toDB(mizuho_tickers_df, 'Datasets/Mizuho_tickers_df.csv')

In [193]:
ticker_df_valid = pd.concat([ticker_df_valid, mizuho_tickers_df[ticker_df_valid.columns]], axis=0)

In [194]:
DL.toDB(ticker_df_valid, 'Datasets/ticker_df_valid.csv')

In [200]:
ticker_df_valid = DL.loadDB('ticker_df_valid.csv')

12:09:58 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/ticker_df_valid.csv


In [197]:
ticker_df_invalid = ticker_df_valid[ticker_df_valid['Ticker'].isna()]
ticker_df_invalid

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
7872,NaN,NaN,CORE SS Equity,SS,SE0002257402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7873,NaN,NaN,GWI1 GY Equity,GY,DE0003304101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7874,NaN,NaN,VET SW Equity,SW,CH0006227612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7875,NaN,NaN,NOFI NO Equity,NO,NO0010387004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7876,NaN,NaN,FCAU US Equity,US,NL0010877643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7877,NaN,NaN,METSO FH Equity,FH,FI0009007835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7878,NaN,NaN,APY US Equity,US,US03755L1044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7879,NaN,NaN,DISCA US Equity,US,US25470F1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7880,NaN,NaN,SNH US Equity,US,US81721M1099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7881,NaN,NaN,MYL US Equity,US,NL0011031208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
universe = DL.loadDB('universe.csv')

12:47:55 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/universe.csv


In [207]:
universe_dup = universe[universe.duplicated(['Ticker'], keep=False)]

In [209]:
universe_dup = universe_dup[['Ticker(old)', 'Ticker']]

In [211]:
ticker_df_valid.head(5)

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,1721,1721.T,1721 JP Equity,JP,JP3305530002,1721.T,Industrials,Construction & Engineering,Capital Goods,Construction & Engineering,NaN,Japan,Japan
1,1878,1878.T,1878 JP Equity,JP,JP3486800000,1878.T,Real Estate,Real Estate Management & Development,Real Estate,Diversified Real Estate Activities,NaN,Japan,Japan
2,1951,1951.T,1951 JP Equity,JP,JP3254200003,1951.T,Industrials,Construction & Engineering,Capital Goods,Construction & Engineering,NaN,Japan,Japan
3,2002,2002.T,2002 JP Equity,JP,JP3676800000,2002.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,NaN,Japan,Japan
4,2267,2267.T,2267 JP Equity,JP,JP3931600005,2267.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,NaN,Japan,Japan


In [210]:
universe_dup

,Ticker(old),Ticker
0,1U1.DE,1U1.DE
1,DRIG.DE,1U1.DE
2,TXG,TXG.O
3,TXG.O,TXG.O
5,ONEM,ONEM.O
...,...,...
4383,7453.T,7453.T
4384,8227.T,8227.T
4385,9409.T,9409.T
4386,9602.T,9602.T


In [234]:
ticker_df_valid = pd.concat([ticker_df_valid, universe_dup], axis=0).drop_duplicates(['Ticker(old)', 'Ticker'])

In [235]:
ticker_df_invalid = ticker_df_valid[ticker_df_valid['Ticker(BBG)'].isna()]
ticker_df_valid = ticker_df_valid[~ticker_df_valid['Ticker(BBG)'].isna()]

ticker_df_valid_isin_dict = ticker_df_valid.set_index('Ticker').to_dict()['ISIN']
ticker_df_valid_bbg_dict = ticker_df_valid.set_index('Ticker').to_dict()['Ticker(BBG)']

In [237]:
ticker_df_invalid['Ticker(BBG)'] = ticker_df_invalid['Ticker'].replace(ticker_df_valid_bbg_dict)
ticker_df_invalid['ISIN'] = ticker_df_invalid['Ticker'].replace(ticker_df_valid_isin_dict)

In [232]:
sectors6 = ek.get_data(list(ticker_df_invalid['ISIN']), 
                      ['TR.GICSSector', 'TR.GICSIndustry', 'TR.GICSIndustryGroup', 'TR.GICSSubIndustry',
                      'CF_EXCHANGE', 'TR.ExchangeCountry', 'TR.UltimateParentCountryHQ'])[0]

ticker_df_invalid = pd.concat([ticker_df_invalid, 
                               sectors6[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                         'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']]], axis=1)

In [240]:
ticker_df_invalid[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                         'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']] = 
sectors6[['GICS Sector Name', 'GICS Industry Name', 'GICS Industry Group Name', 'GICS Sub-Industry Name',
                                         'CF_EXCHANGE', 'Country of Exchange', 'Country of Headquarters']]

In [243]:
ticker_df_valid = pd.concat([ticker_df_valid, ticker_df_invalid[ticker_df_valid.columns]], axis=0)
ticker_df_valid

,Ticker(old),Ticker,Ticker(BBG),Region,ISIN,RIC,GICS Sector Name,GICS Industry Name,GICS Industry Group Name,GICS Sub-Industry Name,CF_EXCHANGE,Country of Exchange,Country of Headquarters
0,1721,1721.T,1721 JP Equity,JP,JP3305530002,1721.T,Industrials,Construction & Engineering,Capital Goods,Construction & Engineering,<NA>,Japan,Japan
1,1878,1878.T,1878 JP Equity,JP,JP3486800000,1878.T,Real Estate,Real Estate Management & Development,Real Estate,Diversified Real Estate Activities,<NA>,Japan,Japan
2,1951,1951.T,1951 JP Equity,JP,JP3254200003,1951.T,Industrials,Construction & Engineering,Capital Goods,Construction & Engineering,<NA>,Japan,Japan
3,2002,2002.T,2002 JP Equity,JP,JP3676800000,2002.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,<NA>,Japan,Japan
4,2267,2267.T,2267 JP Equity,JP,JP3931600005,2267.T,Consumer Staples,Food Products,"Food, Beverage & Tobacco",Packaged Foods & Meats,<NA>,Japan,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,7453.T,7453.T,7453 JP Equity,NaN,JP3976300008,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
4384,8227.T,8227.T,8227 JP Equity,NaN,JP3358200008,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
4385,9409.T,9409.T,9409 JP Equity,NaN,JP3429000007,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
4386,9602.T,9602.T,9602 JP Equity,NaN,JP3598600009,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [244]:
DL.toDB(ticker_df_valid, 'ticker_df_valid2.csv')

In [245]:
price_df = DL.loadDB('price_df.csv')

16:54:03 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/price_df.csv


In [246]:
ticker_df_valid_bbg_dict = ticker_df_valid.set_index('Ticker(old)').to_dict()['Ticker(BBG)']
price_df['Ticker(BBG)'] = price_df['Ticker'].replace(ticker_df_valid_bbg_dict)
price_df['Ticker(BBG)']

0         WEX US Equity
1         FLT US Equity
2        ALLY US Equity
3        TMUS US Equity
4         CNR CN Equity
              ...      
60352     EXX SJ Equity
60353     SBK SJ Equity
60354     NED SJ Equity
60355     VOD SJ Equity
60356     BVT SJ Equity
Name: Ticker(BBG), Length: 60357, dtype: object

In [247]:
DL.toDB(price_df, 'price_df(BBG).csv')

In [248]:
Eikon_update_price(['JP3486800000'])

New ticker: JP3486800000, downloading from Eikon


2022-04-28 18:01:21,833 P[17360] [MainThread 27976] Error with JP3486800000: Invalid RIC
2022-04-28 18:01:21,834 P[17360] [MainThread 27976] JP3486800000: Invalid RIC | 


18:01:21 Position: [<FrameSummary file C:\Users\JayYang\AppData\Local\Temp/ipykernel_17360/1566893619.py, line 47 in Eikon_update_price>, <FrameSummary file C:\Users\JayYang\Anaconda3\lib\site-packages\eikon\time_series.py, line 203 in get_timeseries>] | Error type: <class 'eikon.eikonError.EikonError'> | Error: Error code -1 | JP3486800000: Invalid RIC | 
18:01:21 JP3486800000
18:01:21 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi\Database/Log/20220427/Anomalies_MC_152757.csv created.
